<a href="https://colab.research.google.com/github/faijan-khan/brainbuddy/blob/main/EPICS_VIRTUAL_OUTFIT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

For inferencing ACGPN and SieveNet!

ACGPN repo: https://github.com/switchablenorms/DeepFashion_Try_On

SieveNet repo: https://github.com/levindabhi/SieveNet

This notebook is hard coded for inferencing one image at a time.

Notebook by [Levin Dabhi](https://levindabhi.github.io/)

# ACGPN

In [1]:
!git clone https://github.com/levindabhi/ACGPN.git
%cd ACGPN

Cloning into 'ACGPN'...
remote: Enumerating objects: 161, done.
remote: Counting objects: 100% (22/22), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 161 (delta 20), reused 18 (delta 18), pack-reused 139 (from 1)
Receiving objects: 100% (161/161), 302.85 KiB | 1.17 MiB/s, done.
Resolving deltas: 100% (59/59), done.
/content/ACGPN


In [2]:
!pip install ninja

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 422.9/422.9 kB 9.0 MB/s eta 0:00:00


In [3]:
import gdown
import numpy as np
from PIL import Image
import IPython
import gdown
import os
import sys

from predict_pose import generate_pose_keypoints

In [4]:
'''
gdown.download('https://drive.google.com/uc?id=1tE7hcVFm8Td8kRh5iYRBSDFdvZIkbUIR', 'Data_preprocessing/data.zip', quiet=False)
%cd Data_preprocessing
!unzip data
%cd ..
'''

"\ngdown.download('https://drive.google.com/uc?id=1tE7hcVFm8Td8kRh5iYRBSDFdvZIkbUIR', 'Data_preprocessing/data.zip', quiet=False)\n%cd Data_preprocessing\n!unzip data\n%cd ..\n"

In [5]:
!mkdir Data_preprocessing/test_color
!mkdir Data_preprocessing/test_colormask
!mkdir Data_preprocessing/test_edge
!mkdir Data_preprocessing/test_img
!mkdir Data_preprocessing/test_label
!mkdir Data_preprocessing/test_mask
!mkdir Data_preprocessing/test_pose
!mkdir inputs
!mkdir inputs/img
!mkdir inputs/cloth

In [6]:
%cd pose
!gdown --id 1hOHMFHEjhoJuLEQY0Ndurn5hfiA9mwko
%cd ..

/content/ACGPN/pose
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Failed to retrieve file url:

	Cannot retrieve the public link of the file. You may need to change
	the permission to 'Anyone with the link', or have had many accesses.
	Check FAQ in https://github.com/wkentaro/gdown?tab=readme-ov-file#faq.

You may still be able to access the file from the browser:

	https://drive.google.com/uc?id=1hOHMFHEjhoJuLEQY0Ndurn5hfiA9mwko

but Gdown can't. Please check connections and permissions.
/content/ACGPN


In [7]:
!git clone https://github.com/levindabhi/Self-Correction-Human-Parsing-for-ACGPN.git
!git clone https://github.com/levindabhi/U-2-Net.git

Cloning into 'Self-Correction-Human-Parsing-for-ACGPN'...
remote: Enumerating objects: 769, done.
remote: Counting objects: 100% (203/203), done.
remote: Compressing objects: 100% (109/109), done.
remote: Total 769 (delta 103), reused 94 (delta 94), pack-reused 566 (from 1)
Receiving objects: 100% (769/769), 3.89 MiB | 8.02 MiB/s, done.
Resolving deltas: 100% (184/184), done.
Cloning into 'U-2-Net'...
remote: Enumerating objects: 822, done.
remote: Counting objects: 100% (334/334), done.
remote: Compressing objects: 100% (35/35), done.
remote: Total 822 (delta 312), reused 299 (delta 299), pack-reused 488 (from 1)
Receiving objects: 100% (822/822), 30.71 MiB | 24.58 MiB/s, done.
Resolving deltas: 100% (391/391), done.


In [8]:
#for segmentation mask generation
url = 'https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH'
output = 'lip_final.pth'
gdown.download(url, output, quiet=False)

Downloading...
From (original): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH
From (redirected): https://drive.google.com/uc?id=1k4dllHpu0bdx38J7H28rVVLpU-kOHmnH&confirm=t&uuid=c01032a6-dfec-493e-ab7d-d391091b949d
To: /content/ACGPN/lip_final.pth
100%|██████████| 267M/267M [00:01<00:00, 153MB/s]


'lip_final.pth'

In [9]:
%cd U-2-Net
!mkdir saved_models
!mkdir saved_models/u2net
!mkdir saved_models/u2netp
!gdown --id 1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy -O saved_models/u2netp/u2netp.pth
!gdown --id 1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ -O saved_models/u2net/u2net.pth
import u2net_load
import u2net_run
u2net = u2net_load.model(model_name = 'u2netp')
%cd ..

/content/ACGPN/U-2-Net
mkdir: cannot create directory ‘saved_models’: File exists
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From: https://drive.google.com/uc?id=1rbSTGKAE-MTxBYHd-51l2hMOQPT_7EPy
To: /content/ACGPN/U-2-Net/saved_models/u2netp/u2netp.pth
100% 4.68M/4.68M [00:00<00:00, 32.7MB/s]
/usr/local/lib/python3.11/dist-packages/gdown/__main__.py:140: FutureWarning: Option `--id` was deprecated in version 4.3.1 and will be removed in 5.0. You don't need to pass it anymore to use a file ID.
  warnings.warn(
Downloading...
From (original): https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ
From (redirected): https://drive.google.com/uc?id=1ao1ovG1Qtx4b7EoskHXmi2E9rp5CHLcZ&confirm=t&uuid=807da0fe-ebe9-4a87-8555-2e333263e6ca
To: /content/ACGPN/U-2-Net/saved_models/u2net/u2net.pth
10

In [10]:
!mkdir checkpoints
gdown.download('https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx',output='checkpoints/ACGPN_checkpoints.zip', quiet=False)
%cd checkpoints
!unzip ACGPN_checkpoints
%cd ..

Downloading...
From (original): https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx
From (redirected): https://drive.google.com/uc?id=1UWT6esQIU_d4tUm8cjxDKMhB8joQbrFx&confirm=t&uuid=c343ec9f-d6f4-43c3-bd67-7dddc23a2ccc
To: /content/ACGPN/checkpoints/ACGPN_checkpoints.zip
100%|██████████| 524M/524M [00:03<00:00, 155MB/s]


/content/ACGPN/checkpoints
Archive:  ACGPN_checkpoints.zip
   creating: label2city/
  inflating: label2city/latest_net_G.pth  
  inflating: label2city/latest_net_G1.pth  
  inflating: label2city/latest_net_G2.pth  
  inflating: label2city/latest_net_U.pth  
  inflating: label2city/opt.txt      
/content/ACGPN


In [11]:
!wget -O pose/pose_iter_440000.caffemodel \
  https://github.com/nik1806/Human-Pose-Detection/raw/refs/heads/master/model/coco/pose_iter_440000.caffemodel


--2025-02-27 05:20:29--  https://github.com/nik1806/Human-Pose-Detection/raw/refs/heads/master/model/coco/pose_iter_440000.caffemodel
Resolving github.com (github.com)... 140.82.112.4
Connecting to github.com (github.com)|140.82.112.4|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://media.githubusercontent.com/media/nik1806/Human-Pose-Detection/refs/heads/master/model/coco/pose_iter_440000.caffemodel [following]
--2025-02-27 05:20:29--  https://media.githubusercontent.com/media/nik1806/Human-Pose-Detection/refs/heads/master/model/coco/pose_iter_440000.caffemodel
Resolving media.githubusercontent.com (media.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to media.githubusercontent.com (media.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 209274056 (200M) [application/octet-stream]
Saving to: ‘pose/pose_iter_440000.caffemodel’

pose/pose_iter

In [ ]:
# Install gradio if needed
!pip install gradio

# Add this after all the setup cells (after model downloads and directory creation)
import gradio as gr
import os
import shutil
from PIL import Image
import time
import u2net_run  # Ensure U-2-Net is properly imported

def process_images(human_img, cloth_img):
    try:
        start_time = time.time()

        # Debug: Print current directory
        print("Current working directory:", os.getcwd())

        # Create necessary directories
        os.makedirs('inputs/img', exist_ok=True)
        os.makedirs('inputs/cloth', exist_ok=True)
        os.makedirs('Data_preprocessing/test_img', exist_ok=True)
        os.makedirs('Data_preprocessing/test_color', exist_ok=True)
        os.makedirs('Data_preprocessing/test_edge', exist_ok=True)

        # Fixed file paths
        human_path = 'inputs/img/000001_0.png'
        cloth_path = 'inputs/cloth/000001_1.png'
        test_img_path = 'Data_preprocessing/test_img/000001_0.png'
        test_color_path = 'Data_preprocessing/test_color/000001_1.png'
        test_edge_path = 'Data_preprocessing/test_edge/000001_1.png'

        # Convert and save images
        if human_img is not None:
            img = Image.open(human_img)
            img = img.resize((192,256), Image.BICUBIC)
            img.save(human_path)
            img.save(test_img_path)
            print("Human image saved successfully")

        if cloth_img is not None:
            img = Image.open(cloth_img)
            img = img.resize((192,256), Image.BICUBIC).convert('RGB')
            img.save(cloth_path)
            img.save(test_color_path)
            print("Cloth image saved successfully")

        # Generate edge map using U-2-Net
        print("Generating edge map...")
        u2net_run.infer(u2net, 'Data_preprocessing/test_color', 'Data_preprocessing/test_edge')

        # Remove previous results
        shutil.rmtree('results', ignore_errors=True)
        print("Cleaned previous results")

        # Run segmentation
        print("Running segmentation...")
        !python3 Self-Correction-Human-Parsing-for-ACGPN/simple_extractor.py \
            --dataset 'lip' \
            --model-restore 'lip_final.pth' \
            --input-dir 'Data_preprocessing/test_img' \
            --output-dir 'Data_preprocessing/test_label'

        # Generate pose keypoints
        print("Generating pose keypoints...")
        pose_path = 'Data_preprocessing/test_pose/000001_0_keypoints.json'
        os.makedirs('Data_preprocessing/test_pose', exist_ok=True)
        generate_pose_keypoints(test_img_path, pose_path)

        # Create test pairs file
        with open('Data_preprocessing/test_pairs.txt','w') as f:
            f.write('000001_0.png 000001_1.png')
        print("Created test pairs file")

        # Run inference
        print("Running inference...")
        !python test.py

        # Load and return result
        result_path = 'results/test/try-on/000001_0.png'
        if os.path.exists(result_path):
            print("Processing completed successfully!")
            print(f"Total time: {time.time() - start_time:.2f} seconds")
            return result_path
        else:
            return "Error: No output generated. Check console for details."

    except Exception as e:
        import traceback
        error_msg = f"Error during processing:\n{str(e)}\n\nTraceback:\n{traceback.format_exc()}"
        print(error_msg)
        return error_msg

# Create Gradio interface
interface = gr.Interface(
    fn=process_images,
    inputs=[
        gr.Image(type="filepath", label="Upload Human Image"),
        gr.Image(type="filepath", label="Upload Cloth Image")
    ],
    outputs=gr.Image(label="Result"),
    title="Virtual Try-On Demo",
    description="Upload a human image and a clothing image to see the virtual try-on result",
    allow_flagging="never"
)

# Launch the interface
interface.launch(debug=True)

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. This cell will run indefinitely so that you can see errors and logs. To turn off, set debug=False in launch().
* Running on public URL: https://3a34e22d6742bc8403.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)


Current working directory: /content/ACGPN
Human image saved successfully
Cloth image saved successfully
Generating edge map...
Generating mask for: 000001_1.png
Saving output at Data_preprocessing/test_edge/000001_1.png
Cleaned previous results
Running segmentation...
/usr/local/lib/python3.11/dist-packages/torch/utils/cpp_extension.py:1964: UserWarning: TORCH_CUDA_ARCH_LIST is not set, all archs for visible cards are included for compilation. 
If this is not desired, please set os.environ['TORCH_CUDA_ARCH_LIST'].
  warnings.warn(
/content/ACGPN/Self-Correction-Human-Parsing-for-ACGPN/simple_extractor.py:107: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default valu